In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_table('rfm_xmas19.txt', delimiter = ',')

In [3]:
df.head()

,customer_id,trans_date,tran_amount
0,FM5295,2017-11-11,35
1,FM4768,2019-12-15,39
2,FM2122,2017-11-26,52
3,FM1217,2016-08-16,99
4,FM1850,2018-08-20,78


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  125000 non-null  object
 1   trans_date   125000 non-null  object
 2   tran_amount  125000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.9+ MB


In [5]:
df["trans_date"]=pd.to_datetime(df["trans_date"])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   customer_id  125000 non-null  object        
 1   trans_date   125000 non-null  datetime64[ns]
 2   tran_amount  125000 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 2.9+ MB


In [7]:
df.shape

(125000, 3)

In [8]:
df.head(20)

,customer_id,trans_date,tran_amount
0,FM5295,2017-11-11,35
1,FM4768,2019-12-15,39
2,FM2122,2017-11-26,52
3,FM1217,2016-08-16,99
4,FM1850,2018-08-20,78
5,FM5539,2018-12-26,81
6,FM2724,2016-11-06,93
7,FM5902,2019-10-30,89
8,FM6040,2017-10-08,76
9,FM3802,2018-05-20,75


In [9]:
grouped_customer=df.groupby('customer_id') #We observe that there are multiple customer's with same ID but with different transactions, hence we group the customers based on their ID

In [10]:
latest_transaction_of_each_group=grouped_customer['trans_date'].max()

In [11]:
latest_transaction_of_each_group

customer_id
FM1112   2019-10-14
FM1113   2019-11-09
FM1114   2019-11-12
FM1115   2019-12-05
FM1116   2019-05-25
            ...    
FM8996   2019-09-09
FM8997   2019-03-28
FM8998   2019-09-22
FM8999   2019-04-02
FM9000   2019-11-28
Name: trans_date, Length: 6889, dtype: datetime64[ns]

In [12]:
customer_analysis=pd.DataFrame(latest_transaction_of_each_group)
customer_analysis.head(10)

,trans_date
customer_id,
FM1112,2019-10-14
FM1113,2019-11-09
FM1114,2019-11-12
FM1115,2019-12-05
FM1116,2019-05-25
FM1117,2019-04-02
FM1118,2019-12-14
FM1119,2019-12-05
FM1120,2019-12-06


In [13]:
import datetime as dt
margin=dt.datetime(2019,10,1)
margin

datetime.datetime(2019, 10, 1, 0, 0)

In [14]:
customer_analysis['inactive']=np.where(customer_analysis['trans_date']<=margin,1,0) #the initial margin set for analysis is 1st October, 2019. After this margin, if there's no transaction being made we assume that the customer is inactive

In [15]:
customer_analysis.head(10)

,trans_date,inactive
customer_id,,
FM1112,2019-10-14,0
FM1113,2019-11-09,0
FM1114,2019-11-12,0
FM1115,2019-12-05,0
FM1116,2019-05-25,1
FM1117,2019-04-02,1
FM1118,2019-12-14,0
FM1119,2019-12-05,0
FM1120,2019-12-06,0


In [16]:
inactive_customers=customer_analysis[customer_analysis['inactive']==1]

In [17]:
inactive_customers

,trans_date,inactive
customer_id,,
FM1116,2019-05-25,1
FM1117,2019-04-02,1
FM1123,2019-08-27,1
FM1126,2019-06-18,1
FM1128,2019-09-30,1
...,...,...
FM8994,2019-09-17,1
FM8996,2019-09-09,1
FM8997,2019-03-28,1


In [18]:
inactive_customers['total_transactions']=grouped_customer.size()  #total transaction made by 

C:\Users\somra\AppData\Local\Temp\ipykernel_12624\2552330.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inactive_customers['total_transactions']=grouped_customer.size()


In [19]:
inactive_customers.head(10)

,trans_date,inactive,total_transactions
customer_id,,,
FM1116,2019-05-25,1,13
FM1117,2019-04-02,1,17
FM1123,2019-08-27,1,19
FM1126,2019-06-18,1,19
FM1128,2019-09-30,1,26
FM1129,2019-08-30,1,12
FM1134,2019-08-27,1,16
FM1135,2019-01-15,1,20
FM1139,2019-08-25,1,18


In [20]:
inactive_customers['total_amount']=grouped_customer['tran_amount'].sum() #finding out the total amount
inactive_customers

C:\Users\somra\AppData\Local\Temp\ipykernel_12624\3907852658.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inactive_customers['total_amount']=grouped_customer['tran_amount'].sum()


,trans_date,inactive,total_transactions,total_amount
customer_id,,,,
FM1116,2019-05-25,1,13,857
FM1117,2019-04-02,1,17,1185
FM1123,2019-08-27,1,19,1331
FM1126,2019-06-18,1,19,1165
FM1128,2019-09-30,1,26,1921
...,...,...,...,...
FM8994,2019-09-17,1,16,749
FM8996,2019-09-09,1,13,582
FM8997,2019-03-28,1,14,543


In [21]:
inactive_customers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2698 entries, FM1116 to FM8999
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   trans_date          2698 non-null   datetime64[ns]
 1   inactive            2698 non-null   int32         
 2   total_transactions  2698 non-null   int64         
 3   total_amount        2698 non-null   int64         
dtypes: datetime64[ns](1), int32(1), int64(2)
memory usage: 94.9+ KB


In [22]:
inactive_customers.describe()

,trans_date,inactive,total_transactions,total_amount
count,2698,2698.0,2698.000000,2698.000000
mean,2019-07-08 23:32:46.790214912,1.0,16.801705,1063.872498
min,2017-08-09 00:00:00,1.0,4.000000,149.000000
25%,2019-06-07 06:00:00,1.0,13.000000,644.250000
50%,2019-08-06 00:00:00,1.0,17.000000,1101.500000
75%,2019-09-08 00:00:00,1.0,20.000000,1400.000000
max,2019-10-01 00:00:00,1.0,38.000000,2647.000000
std,NaN,0.0,5.102106,459.111630


In [23]:
#Normalizing by min-max Scaling method 
inactive_customers['scaled_transaction']=(inactive_customers['total_transactions']-inactive_customers['total_transactions'].min())\
/(inactive_customers['total_transactions'].max()-inactive_customers['total_transactions'].min())

C:\Users\somra\AppData\Local\Temp\ipykernel_12624\936305320.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inactive_customers['scaled_transaction']=(inactive_customers['total_transactions']-inactive_customers['total_transactions'].min())\


In [24]:
inactive_customers.head()

,trans_date,inactive,total_transactions,total_amount,scaled_transaction
customer_id,,,,,
FM1116,2019-05-25,1,13,857,0.264706
FM1117,2019-04-02,1,17,1185,0.382353
FM1123,2019-08-27,1,19,1331,0.441176
FM1126,2019-06-18,1,19,1165,0.441176
FM1128,2019-09-30,1,26,1921,0.647059


In [25]:
inactive_customers['scaled_amount']=(inactive_customers['total_amount']-inactive_customers['total_amount'].min())\
/(inactive_customers['total_amount'].max()-inactive_customers['total_amount'].min())

C:\Users\somra\AppData\Local\Temp\ipykernel_12624\2033467758.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inactive_customers['scaled_amount']=(inactive_customers['total_amount']-inactive_customers['total_amount'].min())\


In [26]:
inactive_customers.head()

,trans_date,inactive,total_transactions,total_amount,scaled_transaction,scaled_amount
customer_id,,,,,,
FM1116,2019-05-25,1,13,857,0.264706,0.283427
FM1117,2019-04-02,1,17,1185,0.382353,0.414732
FM1123,2019-08-27,1,19,1331,0.441176,0.473179
FM1126,2019-06-18,1,19,1165,0.441176,0.406725
FM1128,2019-09-30,1,26,1921,0.647059,0.709367


In [27]:
#creating scoring column ie. the weighted sum model. (1/2 * scaled_transactions) + (1/2 * scaled_amount)
inactive_customers['score']=((0.5*inactive_customers['scaled_transaction'])+(0.5*inactive_customers['scaled_amount']))*100

C:\Users\somra\AppData\Local\Temp\ipykernel_12624\735432655.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inactive_customers['score']=((0.5*inactive_customers['scaled_transaction'])+(0.5*inactive_customers['scaled_amount']))*100


In [28]:
inactive_customers.head()

,trans_date,inactive,total_transactions,total_amount,scaled_transaction,scaled_amount,score
customer_id,,,,,,,
FM1116,2019-05-25,1,13,857,0.264706,0.283427,27.406631
FM1117,2019-04-02,1,17,1185,0.382353,0.414732,39.854236
FM1123,2019-08-27,1,19,1331,0.441176,0.473179,45.717751
FM1126,2019-06-18,1,19,1165,0.441176,0.406725,42.395093
FM1128,2019-09-30,1,26,1921,0.647059,0.709367,67.821316


In [29]:
final_margin=dt.datetime(2019,1,1)
final_margin

datetime.datetime(2019, 1, 1, 0, 0)

In [30]:
final_customer_data=inactive_customers[inactive_customers['trans_date']>=final_margin]
final_customer_data

,trans_date,inactive,total_transactions,total_amount,scaled_transaction,scaled_amount,score
customer_id,,,,,,,
FM1116,2019-05-25,1,13,857,0.264706,0.283427,27.406631
FM1117,2019-04-02,1,17,1185,0.382353,0.414732,39.854236
FM1123,2019-08-27,1,19,1331,0.441176,0.473179,45.717751
FM1126,2019-06-18,1,19,1165,0.441176,0.406725,42.395093
FM1128,2019-09-30,1,26,1921,0.647059,0.709367,67.821316
...,...,...,...,...,...,...,...
FM8994,2019-09-17,1,16,749,0.352941,0.240192,29.656667
FM8996,2019-09-09,1,13,582,0.264706,0.173339,21.902228
FM8997,2019-03-28,1,14,543,0.294118,0.157726,22.592191


In [31]:
final_customer_data.sort_values('score', ascending=False, inplace=True)
final_customer_data

C:\Users\somra\AppData\Local\Temp\ipykernel_12624\305713916.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_customer_data.sort_values('score', ascending=False, inplace=True)


,trans_date,inactive,total_transactions,total_amount,scaled_transaction,scaled_amount,score
customer_id,,,,,,,
FM4320,2019-09-26,1,38,2647,1.000000,1.000000,100.000000
FM3799,2019-07-16,1,36,2513,0.941176,0.946357,94.376678
FM5752,2019-09-28,1,33,2612,0.852941,0.985989,91.946498
FM3805,2019-09-11,1,35,2453,0.911765,0.922338,91.705129
FM4074,2019-09-05,1,34,2462,0.882353,0.925941,90.414685
...,...,...,...,...,...,...,...
FM8224,2019-09-21,1,5,187,0.029412,0.015212,2.231197
FM8841,2019-05-10,1,5,181,0.029412,0.012810,2.111101
FM8234,2019-09-21,1,5,157,0.029412,0.003203,1.630716


In [32]:
#Now the organization only has $1000 to spare for coupons. Now we calculate the coupon price based on 30%
coupon=df['tran_amount'].mean()*0.30
coupon

19.4975736

In [33]:
#Since we have only $1000 where each customer shall get $19, we find total how many customers can get this coupon
customers_coupons=1000/19
customers_coupons

52.63157894736842

In [34]:
eligible_customers=final_customer_data.head(52)

In [35]:
eligible_customers.drop(columns=['trans_date','inactive','total_transactions','total_amount','scaled_transaction','scaled_amount'], inplace=True)

C:\Users\somra\AppData\Local\Temp\ipykernel_12624\971515017.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eligible_customers.drop(columns=['trans_date','inactive','total_transactions','total_amount','scaled_transaction','scaled_amount'], inplace=True)


In [76]:
eligible_customers.shape

(52, 1)

In [78]:
eligible_customers

,score
customer_id,
FM4320,100.000000
FM3799,94.376678
FM5752,91.946498
FM3805,91.705129
FM4074,90.414685
FM1215,89.883672
FM2620,89.843640
FM1580,86.281967
FM2951,85.872227
